In [200]:
import requests
import json
import pandas as pd
from datetime import timedelta, date, datetime
import time

addresses = set(
    [
        "0xef6d69922bc2d038cb508086846524f8011c4a74",
        "0xd387a6e4e84a6c86bd90c158c6028a58cc8ac459",
        "0xe1d29d0a39962a9a8d2a297ebe82e166f8b8ec18",
        "0xd5a5c2ef8977ab40d920417018145508251a07c0",
        "0x1b2965dc3b1697dd10b7126ec7393d79cda2cf91",
        "0x78f0269f5b1ca914c8c58993f41181f89e212b27",
        "0x8dbbca57ea56290efa14d835bbfd34faf1d89753",
        "0x53dda0a69992b87e51f0f14b1337281f09a2c8da",
        "0x63a9dbce75413036b2b778e670aabd4493aaf9f3",
        "0x3612b2e93b49f6c797066ca8c38b7f522b32c7cb",
        "0xebf02c6e12c3ee119abba161c40bfeead0a06b15",
        "0xf6bc89cc57e33c4e8053c4d8e63b70bf8c47e1a1",
        "0x530cf036ed4fa58f7301a9c788c9806624cefd19",
        "0x79ecd68a6bcdf40f40bf0dc0c1fcc962aa8c6213",
        "0x6b67623ff56c10d9dcfc2152425f90285fc74ddd",
        "0xe10824fd1ffb9951215025eb26cda42d2d8d9c4c",
        "0x9e199d8a3a39c9892b1c3ae348a382662dcbaa12",
        "0xd5a9c4a92dde274e126f82b215fccb511147cd8e",
        "0xbff79922fcbf93f9c30abb22322b271460c6bebb",
        "0xb53349160e38739b37e4bbfcf950ed26e26fcb41",
    ]
)
events = []
a = None
url = "https://api.opensea.io/api/v1/events"
for address in addresses:
    querystring = {
        "only_opensea": "false",
        "offset": "0",
        "limit": "200",
        "account_address": address,
        "event_type": "successful",
    }
    headers = {"Accept": "application/json"}
    response = requests.request("GET", url, headers=headers, params=querystring)
    events += json.loads(response.text)["asset_events"]
    time.sleep(10)


In [201]:
buy_sell_dict = {"timestamp": [], "asset_name": [], "type": [], "price": []}
for event in events:
    if event["asset"] is None:
        continue
    buy_sell_dict["timestamp"].append(
        datetime.strptime(event["transaction"]["timestamp"], "%Y-%m-%dT%H:%M:%S")
    )
    buy_sell_dict["asset_name"].append(event["asset"]["asset_contract"]["name"])
    buy_sell_dict["type"].append(
        "sell" if event["winner_account"]["address"] not in addresses else "buy"
    )
    buy_sell_dict["price"].append(float(event["total_price"]) / 10e17)

buy_sell_df = pd.DataFrame.from_dict(buy_sell_dict)

buy_sell_df


,timestamp,asset_name,type,price
0,2021-09-04 20:33:58,CyberKongz VX,buy,2.500000
1,2021-09-04 20:11:34,CyberKongz VX,buy,4.000000
2,2021-09-04 20:09:02,CyberKongz VX,buy,0.669000
3,2021-09-04 20:08:02,CyberKongz VX,buy,0.669000
4,2021-09-04 16:29:41,Art Blocks,buy,6.460416
...,...,...,...,...
3618,2021-08-30 07:18:23,OpenSea Collection,buy,0.250000
3619,2021-08-30 07:17:39,OpenSea Collection,buy,0.250000
3620,2021-08-30 07:16:43,OpenSea Collection,buy,0.250000
3621,2021-08-30 07:14:09,CryptoKitties,buy,0.400000


In [202]:
top_buy = (
    buy_sell_df[
        (buy_sell_df["timestamp"] >= datetime.now() - timedelta(days=1))
        & (buy_sell_df["type"] == "buy")
    ][["asset_name", "type"]]
    .groupby(["asset_name", "type"])
    .size()
    .reset_index(name="counts")
    .sort_values("counts", ascending=False)
)
top_sell = (
    buy_sell_df[
        (buy_sell_df["timestamp"] >= datetime.now() - timedelta(days=1))
        & (buy_sell_df["type"] == "sell")
    ][["asset_name", "type"]]
    .groupby(["asset_name", "type"])
    .size()
    .reset_index(name="counts")
    .sort_values("counts", ascending=False)
)


In [205]:
top_buy

,asset_name,type,counts
8,WORD,buy,14
3,Frontier,buy,5
2,CyberKongz VX,buy,4
0,Art Blocks,buy,2
6,OpenSea Collection,buy,2
1,Ciphersquares,buy,1
4,Loot,buy,1
5,MATH,buy,1
7,Rarible 1155,buy,1
9,sLoot,buy,1


In [206]:
top_sell

,asset_name,type,counts
20,RKLSneakers,sell,43
5,CryptoKitties,sell,26
21,Rogue Society Bots,sell,20
4,Crypto-Pills by Micha Klein,sell,13
15,MonkeyBet,sell,11
1,Art Blocks,sell,8
22,RumbleKongLeague,sell,4
29,n,sell,4
17,One Hour Time Pieces,sell,3
3,Bloot,sell,2


In [194]:
hist_dict = {'timestamp': [], 'asset_id': [], 'asset_name': [], 'token_id': [], 'seller': [], 'buyer': [], 'price': []}
for event in events:
    if event['asset'] is None:
        continue
    hist_dict['timestamp'].append(event['transaction']['timestamp'] if event['transaction'] else event['created_date'])
    hist_dict['asset_id'].append(event['asset']['id'])
    hist_dict['asset_name'].append(event['asset']['asset_contract']['name'])
    hist_dict['token_id'].append(event['asset']['token_id'])
    hist_dict['seller'].append(event['seller']['user']['username'] if event['seller']['user'] is not None else None)
    hist_dict['buyer'].append(event['winner_account']['user']['username'] if event['winner_account']['user'] is not None else None)
    hist_dict['price'].append(float(event['total_price']) / 10e17)

hist_df = pd.DataFrame.from_dict(hist_dict)

,asset_name,type
0,Art Blocks,sell
1,Art Blocks,sell
2,Art Blocks,sell
3,Art Blocks,sell
4,Art Blocks,sell
5,Art Blocks,sell
6,Art Blocks,sell
196,Art Blocks,sell
197,BASTARD GAN PUNKS V2,sell
198,Meebits,sell
